## NC Water Supply Plan Scraper
The NC DWR provides access to local water supply plans at this link: https://www.ncwater.org/Water_Supply_Planning/Local_Water_Supply_Plan/. Individual plans are listed by the public water supplier ID (PWSID) for the years 1997 to 2017. For example, Durham's 2016 report is found [here](https://www.ncwater.org/Water_Supply_Planning/Local_Water_Supply_Plan/report.php?pwsid=03-32-010&year=2017).

This script is a work in progress, aiming to scrape the data from each report. 

In [17]:
#Import libraries
import urllib
import pandas as pd
import re
from bs4 import BeautifulSoup as bs

In [10]:
#The url for the report page
theURL = 'https://www.ncwater.org/Water_Supply_Planning/Local_Water_Supply_Plan/report.php?pwsid=03-32-010&year=2017'

In [11]:
#Read the URL 
with urllib.request.urlopen(theURL) as response:
   html = response.read()

In [8]:
#Convert to 'soup
soup = bs(html,'html5lib')

In [85]:
#Create a dictionary of Tables (which have <h6> tags)
tableDict = {}
for t in soup.body.findAll('h6'):
    print(t.text)
    tableDict[t.text]=t

Contact Information
Distribution System
Programs
Water Conservation
Service Area
Water Use by Type
Water Sales
Monthly Withdrawals & Purchases
Surface Water Sources
Surface Water Sources (continued)
Water Purchases From Other Systems
Water Treatment Plants
Monthly Discharges
Wastewater Permits
Wastewater Interconnections
Projections
Future Supply Sources
 Demand v/s Percent of Supply
Additional Information


With the tables nabbed, we need code to handle each individually as structures vary

In [92]:
#Distribution System 
theKey = 'Distribution System'
theHTML = tableDict[theKey].find_next_sibling().prettify()
df = pd.read_html(theHTML)[0]
df.columns = df.iloc[0]
df.drop(0,axis=0,inplace=True)
df

,Line Type,Size Range (Inches),Estimated % of lines
1,Asbestos Cement,6-8,0.21 %
2,Cast Iron,6-42,22.46 %
3,Ductile Iron,6-72,76.39 %
4,Other,6-72,0.94 %


In [97]:
#Water use information
theKey = 'Service Area'
theHTML = tableDict[theKey].find_next_sibling().prettify()

dfSubBasin = pd.read_html(theHTML)[1]
dfSubBasin.columns = df.iloc[0]
dfSubBasin.drop(0,axis=0,inplace=True)

dfCounty = pd.read_html(theHTML)[2]
dfCounty.columns = df.iloc[0]
dfCounty.drop(0,axis=0,inplace=True)


1,Durham,100 %
1,Durham,100 %


In [99]:
#Water Use by Type
theKey = 'Water Use by Type'
theHTML = tableDict[theKey].find_next_sibling().prettify()
df = pd.read_html(theHTML)[0]
df.columns = df.iloc[0]
df.drop(0,axis=0,inplace=True)
df

,Type of Use,Metered Connections,Metered Average Use (MGD),Non-Metered Connections,Non-Metered Estimated Use (MGD)
1,Residential,86834,13.7000,0,0.0000
2,Commercial,5811,5.8000,0,0.0000
3,Industrial,120,2.2200,0,0.0000
4,Institutional,1332,3.5400,0,0.0000


In [126]:
#Distribution System 
theKey = 'Water Sales'
theHTML = tableDict[theKey].find_next_sibling().prettify()
df = pd.read_html(theHTML)[0]
cols = ['Purchaser',
        'PWSID',
        'Average  Daily Sold  (MGD)',
        'Days  Used',
        'Contract:MGD',
        'Contract:Expiration',
        'Contract:Recurring',
        'Required to comply with water  use restrictions?',
        'Pipe Size(s)  (Inches)',
        'Use  Type']
df.columns = cols
df.drop([0,1],axis=0,inplace=True)
df

,Purchaser,PWSID,Average Daily Sold (MGD),Days Used,Contract:MGD,Contract:Expiration,Contract:Recurring,Required to comply with water use restrictions?,Pipe Size(s) (Inches),Use Type
2,Cary,03-92-020,1.9100,15,NaN,2028,Yes,Yes,16.0,Emergency
3,Chatham Co,03-19-126,0.1790,365,4.0000,2028,No,Yes,16.0,Regular
4,City of Raleigh,03-92-010,0.0000,0,1.3000,2026,Yes,No,16.0,Emergency
5,Hillsborough,03-68-015,0.0000,0,NaN,2008,Yes,Yes,16.0,Emergency
6,Orange-Alamance,03-68-020,0.0000,0,NaN,NaN,No,Yes,NaN,Emergency
7,OWASA,03-68-010,4.1320,3,NaN,2029,Yes,Yes,12.0,Emergency


And so on....

And then to loop through each year and each PWSID to create annual tables from each...